**Final project for Statistical Learning 760.**  \\

---

Train a Conv. Net. on CIFAR 10 Data and compare results with a random forest.

In [0]:
#Imports
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from keras.utils import print_summary, to_categorical
import matplotlib.pyplot as plt
import sys
import os

#Test if GPU is working -> '/device:GPU:0'
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
# loading CIFAR 10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# normalize images
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

# make CIFAR 10 labels categorical
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
print(f"New y_test shape: {y_test.shape}")
print(f"y_test sample: {y_test[0:1,]}")

In [0]:
# CNN MODEL

model_name = 'cifar_10_cnn'

# Model type
model = Sequential()
model.reset_states()

#Dropout rate
dropout_rate = 0.5

#Input
print(f"Input Shape: {x_train.shape[1:]}")

# CNN Layer 1
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same',  use_bias=True, bias_initializer='zeros', kernel_initializer='he_normal', activation='relu', input_shape=x_train.shape[1:]))
model.add(BatchNormalization(center=False, scale=False))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Dropout(dropout_rate))

# CNN Layer 4
model.add(Conv2D(64, (3,3), strides=(1,1), padding='valid', use_bias=True, bias_initializer='zeros', kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization(center=False, scale=False))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Dropout(dropout_rate))

# CNN Layer 6
model.add(Conv2D(128, (3,3), strides=(1,1), padding='valid', use_bias=True, bias_initializer='zeros', kernel_initializer='he_normal', activation='relu'))
model.add(BatchNormalization(center=False, scale=False))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Dropout(dropout_rate))

# Flatten
model.add(Flatten())          
          
# Fully Connected Later 5
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', bias_initializer='zeros' , name='fc_1')) 
model.add(BatchNormalization(center=False, scale=False))
model.add(Dropout(dropout_rate))

# Fully Connected Later 6
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', bias_initializer='zeros', name='fc_2')) 
model.add(BatchNormalization(center=False, scale=True))
model.add(Dropout(dropout_rate))

# OUTPUT LAYER
model.add(Dense(num_classes, activation='softmax'))

# Summary 
model.summary()

Building CNN model.

Train the model.

In [0]:
# TRAINING

# Optimizer
optType = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0)
# optType = SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)

# Configure model
model.compile(optimizer=optType, 
              loss='categorical_crossentropy',
              metrics=['accuracy']) #categorical_accuracy

# Train the model
history = model.fit(x=x_train, y=y_train,
          batch_size=128, 
          epochs=100, 
          verbose=2, 
          validation_split=0.1,
          shuffle=True)

Test the model and gather metrics

In [0]:
# TESTING AND METRICS

# Test loss and accuracy
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


# Accuracy plot
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)


# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

In [0]:
#Save Model as yaml file
from keras.models import load_model
from google.colab import files

# Download File
my_file_name = "cifar_10_model.h5"
model.save(my_file_name)
files.download(my_file_name)